In [1]:
!pip install dfply
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser
import re
import math
from dfply import *

In [2]:
transfers_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/Transfers_Full.csv"
transfers = pd.read_csv(transfers_path, index_col = 0)

def remove_chars(string):
  return string.strip()

def clean_money(string):
  ammount = string
  try:
    if("free" in string.lower()):
      ammount = 0.0
    elif("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def is_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return result

def no_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return not(result)

def clean_loans(string):
  ammount = 0.0
  try:
    if("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def call_nan(string):
  return math.isnan(string)


transfers = transfers.drop_duplicates()
transfers["Name"] = transfers["Name"].apply(remove_chars)
transfers["From"] = transfers["From"].apply(remove_chars)
transfers["To"] = transfers["To"].apply(remove_chars)
transfers = transfers[transfers["Player_Id"] != "\nAtalanta "]
transfers.Player_Id = transfers.Player_Id.astype(int)
transfers.tm_Id = transfers.tm_Id.astype(int)
transfers["Date"] = pd.to_datetime(transfers["Date"], errors="coerce")

loans = transfers[transfers["Fee"].apply(is_loan)]
transfers = transfers[(transfers["Fee"].apply(no_loan))]
transfers["Market Value"] = transfers["Market Value"].apply(clean_money)
transfers["Market Value"] = pd.to_numeric(transfers["Market Value"], errors="coerce")
transfers["Fee"] = transfers["Fee"].apply(clean_money)
transfers["Fee"] = pd.to_numeric(transfers["Fee"], errors="coerce")
transfers["Loan"] = False

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
loans["Market Value"] = loans["Market Value"].apply(clean_money)
loans["Market Value"] = pd.to_numeric(loans["Market Value"], errors="coerce")
loans["Fee"] = loans["Fee"].apply(clean_money)
loans["Fee"] = pd.to_numeric(loans["Fee"], errors="coerce")
loans["Loan"] = True
loans.fillna(value = 0, inplace=True)

In [0]:
transfers = transfers.append(loans)
transfers["Season"] = transfers["Season"].astype('category')

In [0]:
transfers.columns = ['Player_Id', 'tm_Id', 'Name', 'From', 'To', 'Fee', 'M_V',
       'Season', 'Date', 'Loan']

In [0]:
# transfers.drop("Player_Id", axis = 1, inplace = True)
transfers = (transfers >>
             arrange(X.Player_Id) >>
             mask(X.Fee.notna()) >>
             drop(X.Player_Id))
# transfers = transfers[transfers['Fee'].notna()]

In [7]:
print(transfers.dtypes)
transfers.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.csv")

tm_Id        int64
Name        object
From        object
To          object
Fee        float64
M_V        float64
Season    category
Date        object
Loan          bool
dtype: object


In [8]:
transfers.head(20)

,tm_Id,Name,From,To,Fee,M_V,Season,Date,Loan
17259,17259,Manuel Neuer,FC Schalke 04,Bayern Munich,30000000.0,28000000.0,11/12,2011-07-01 00:00:00,False
40680,40680,Sven Ulreich,VfB Stuttgart,Bayern Munich,3500000.0,3500000.0,15/16,2015-07-01 00:00:00,False
40680,40680,Sven Ulreich,Schornbach Yth.,Stuttgart Yth.,0.0,NaN,98/99,1998-07-01 00:00:00,False
40680,40680,Sven Ulreich,Lichtenw. Yth.,Schornbach Yth.,0.0,NaN,94/95,1994-07-01 00:00:00,False
336307,336307,Christian Früchtl,Deggendorf Yth.,FC Bayern Münche,0.0,NaN,14/15,2014-07-01 00:00:00,False
317444,317444,Ron-Thorben Hoffmann,Hertha BSC Yth.,RB Leipzig Yth.,0.0,NaN,13/14,2013-07-01 00:00:00,False
281963,281963,Lucas Hernández,CF Rayo Majadaho,Atlético Youth,0.0,NaN,07/08,2007-07-01 00:00:00,False
281963,281963,Lucas Hernández,Atlético Madrid,Bayern Munich,80000000.0,70000000.0,19/20,2019-07-01 00:00:00,False
166601,166601,Niklas Süle,TSG Hoffenheim,Bayern Munich,20000000.0,25000000.0,17/18,2017-07-01 00:00:00,False
166601,166601,Niklas Süle,Darmstadt Yth.,Hoffenheim Yth.,0.0,NaN,09/10,2010-01-01 00:00:00,False
